# Лабораторная работа № 5

## Распознавание объектов на фотографиях

Задачи:

   1. Ознакомиться со сверточными нейронными сетями
   2. Изучить построение модели в Keras в функциональном виде
   3. Изучить работу слоя разреживания (Dropout)

# Цель работы:
Распознавание объектов на фотографиях (Object Recognition in Photographs)
CIFAR-10 (классификация небольших изображений по десяти классам: самолет,
автомобиль, птица, кошка, олень, собака, лягушка, лошадь, корабль и грузовик).

In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten
from keras.utils import np_utils
import numpy as np

In [ ]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # использование ядра 3x3 
pool_size = 2 #   использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 #  вероятность 0,25
drop_prob_2 = 0.5 #  в плотном слое вероятность 0,5
hidden_size = 512 # в плотном слое 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() 
num_train, depth, height, width = X_train.shape 
num_test = X_test.shape[0] 
num_classes = np.unique(y_train).shape[0]
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train)
X_test /= np.max(X_train) 

Y_train = np_utils.to_categorical(y_train, num_classes) 
Y_test = np_utils.to_categorical(y_test, num_classes) 


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)  #Для предотвращения переобучения
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding - сохранять
#размер исходного изображения, рисунок дополняется нулями по краям. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=0, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

170508288/170498071 [==============================] - 154s 1us/step


In [ ]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # использование ядра 3x3 
pool_size = 2 # использование объединения 2х2 
conv_depth_1 = 32 #  32 ядра на слой преобразования
conv_depth_2 = 64 # переход на 64 после первого уровня объединения
drop_prob_1 = 0.25 # выбывает после объединения с вероятностью 0,25   мы будем применять dropout после каждого слоя подвыборки, а также после полносвязного слоя;
drop_prob_2 = 0.5 # выпадают в плотном слое с вероятностью 0,5
hidden_size = 512 # в полносвязном  слое будет 512 нейронов 

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # извлечение данных CIFAR-10
num_train, depth, height, width = X_train.shape # в CIFAR-10 содержится 50000 обучающих примеров
num_test = X_test.shape[0] # в CIFAR-10 содержится 10000 тестовых примеров
num_classes = np.unique(y_train).shape[0] # существует 10 классов изображений
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')

X_train /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]
X_test /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]

Y_train = np_utils.to_categorical(y_train, num_classes) # кодирование тренировачных данных
Y_test = np_utils.to_categorical(y_test, num_classes) # кодирование тестировочных данных


inp = Input(shape=(depth, height, width)) # обратите внимание, глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
#drop_1 = Dropout(drop_prob_1)(pool_1)  #Регуляризация – это любая модификация алгоритма обучения, предпринятая с целью уменьшить его ошибку обобщения, не уменьшая ошибку обучения.
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(pool_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding='same' входное изображение должно иметь нулевое заполнение, чтобы вывод в свертке не отличался по размеру от ввода. 
#drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(pool_2)
hidden = Dense(hidden_size, activation='relu')(flat)
#drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(hidden)

model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1) # verbose=1 - индикатор выполнения
#model.evaluate прогнозирует значения и вычисляет потери и все прикрепленные метрики к модели по заданному набору данных. 
model.evaluate(X_test, Y_test, verbose=1) 

In [ ]:
batch_size = 32 
num_epochs = 200 
kernel_size = 3 # мы будем использовать ядра 3x3 повсюду
pool_size = 2 # мы будем использовать объединение 2х2 повсюду
conv_depth_1 = 64 # первоначально у нас будет 32 ядра на слой преобразования
conv_depth_2 = 128 # переключение на 64 после первого уровня объединения
drop_prob_1 = 0.25 # выбывает после объединения с вероятностью 0,25
drop_prob_2 = 0.5 # выпадают в плотном слое с вероятностью 0,5
hidden_size = 512 # в плотном слое будет 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # извлечение данных CIFAR-10
num_train, depth, height, width = X_train.shape # в CIFAR-10 содержится 50000 обучающих примеров
num_test = X_test.shape[0] # в CIFAR-10 содержится 10000 тестовых примеров
num_classes = np.unique(y_train).shape[0] # существует 10 классов изображений
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]
X_test /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]

Y_train = np_utils.to_categorical(y_train, num_classes) # кодирование тренировачных данных
Y_test = np_utils.to_categorical(y_test, num_classes) # кодирование тестировочных данных


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding='same' входное изображение должно иметь нулевое заполнение, чтобы вывод в свертке не отличался по размеру от ввода. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

In [ ]:
batch_size = 32 
num_epochs = 200 
kernel_size = 2 # мы будем использовать ядра 3x3 повсюду
pool_size = 2 # мы будем использовать объединение 2х2 повсюду
conv_depth_1 = 32 # первоначально у нас будет 32 ядра на слой преобразования
conv_depth_2 = 64 # переключение на 64 после первого уровня объединения
drop_prob_1 = 0.25 # выбывает после объединения с вероятностью 0,25
drop_prob_2 = 0.5 # выпадают в плотном слое с вероятностью 0,5
hidden_size = 512 # в плотном слое будет 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # извлечение данных CIFAR-10
num_train, depth, height, width = X_train.shape # в CIFAR-10 содержится 50000 обучающих примеров
num_test = X_test.shape[0] # в CIFAR-10 содержится 10000 тестовых примеров
num_classes = np.unique(y_train).shape[0] # существует 10 классов изображений
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]
X_test /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]

Y_train = np_utils.to_categorical(y_train, num_classes) # кодирование тренировачных данных
Y_test = np_utils.to_categorical(y_test, num_classes) # кодирование тестировочных данных


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding='same' входное изображение должно иметь нулевое заполнение, чтобы вывод в свертке не отличался по размеру от ввода. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 

In [ ]:
batch_size = 32 
num_epochs = 200 
kernel_size = 5 # мы будем использовать ядра 3x3 повсюду
pool_size = 2 # мы будем использовать объединение 2х2 повсюду
conv_depth_1 = 32 # первоначально у нас будет 32 ядра на слой преобразования
conv_depth_2 = 64 # переключение на 64 после первого уровня объединения
drop_prob_1 = 0.25 # выбывает после объединения с вероятностью 0,25
drop_prob_2 = 0.5 # выпадают в плотном слое с вероятностью 0,5
hidden_size = 512 # в плотном слое будет 512 нейронов

(X_train, y_train), (X_test, y_test) = cifar10.load_data() # извлечение данных CIFAR-10
num_train, depth, height, width = X_train.shape # в CIFAR-10 содержится 50000 обучающих примеров
num_test = X_test.shape[0] # в CIFAR-10 содержится 10000 тестовых примеров
num_classes = np.unique(y_train).shape[0] # существует 10 классов изображений
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]
X_test /= np.max(X_train) # Нормализуйте данные до диапазона [0, 1]

Y_train = np_utils.to_categorical(y_train, num_classes) # кодирование тренировачных данных
Y_test = np_utils.to_categorical(y_test, num_classes) # кодирование тестировочных данных


inp = Input(shape=(depth, height, width)) # N.B. глубина занимает первое место в Керасе
# Conv [32] -> Conv [32] -> Pool (with dropout on the pooling layer)
conv_1 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(inp) #устарело
conv_2 = Convolution2D(conv_depth_1, kernel_size, kernel_size,
padding='same', activation='relu')(conv_1)
pool_1 = MaxPooling2D(pool_size=(pool_size, pool_size))(conv_2)
drop_1 = Dropout(drop_prob_1)(pool_1)
# Conv [64] -> Conv [64] -> Pool (with dropout on the pooling layer)
conv_3 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(drop_1)
conv_4 = Convolution2D(conv_depth_2, kernel_size, kernel_size,
padding='same', activation='relu')(conv_3)
pool_2 = MaxPooling2D(pool_size=(pool_size, pool_size),padding='same')(conv_4) # padding='same' входное изображение должно иметь нулевое заполнение, чтобы вывод в свертке не отличался по размеру от ввода. 
drop_2 = Dropout(drop_prob_1)(pool_2)
#Теперь выровняйте до 1D, примените Плотный -> ReLU (с выпадением) -> softmax
flat = Flatten()(drop_2)
hidden = Dense(hidden_size, activation='relu')(flat)

drop_3 = Dropout(drop_prob_2)(hidden)
out = Dense(num_classes, activation='softmax')(drop_3)
model = Model(inputs=inp, outputs=out) # Чтобы определить модель, просто укажите ее входные и выходные слои
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy']) 
model.fit(X_train, Y_train, batch_size=batch_size, epochs=num_epochs, verbose=1, validation_split=0.1) # 
model.evaluate(X_test, Y_test, verbose=1) 